# Imports

In [2]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments)
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import pandas as pd
from trl import SFTTrainer
from peft.tuners.lora import LoraLayer
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--------------

# Funções

## Carregamento modelo

In [3]:
def create_and_prepare_model(model_name):
    compute_dtype = getattr(torch, "float16")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ],
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, peft_config, tokenizer


## Inferencia

In [7]:
def inference(model, text):
    text_token = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors='pt'
    )
    text_token = text_token.to('cuda:0')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(
            input_ids = text_token.input_ids, 
            max_new_tokens=300,
            # temperature=0.1,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

# Executando

In [5]:
model_name = 'tiiuae/falcon-7b-instruct'
model, peft_config, tokenizer = create_and_prepare_model(model_name)
model = PeftModel.from_pretrained(model, './results/qlora_falcon7b2023-07-12T02:29:08.346680/checkpoint-100')
gen_input = lambda x: f'''\
Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
{x}
###
'''               

Loading checkpoint shards: 100%|██████████| 2/2 [01:36<00:00, 48.45s/it]


In [12]:
model.config.use_cache=False

## Inferencias

In [8]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Alimentação caríssima para um péssimo atendimento.. super mal atendido
###
Prezada cliente,

Lamentamos profundamente pelo ocorrido. Vamos avaliar o atendimento e tomar medidas para garantir que isso não aconteça novamente. 

Pedimos desculpas pelo constrangimento e falta de atenção que você recebeu durante sua visita. Vamos trabalhar para melhorar nossa comunicação e garantir que todos os clientes se sentem bem recebidos aqui.

Quanto ao problema com a falta de atendimento, vamos avaliar a equipe de atendimento e buscar uma solução para garantir que isso não aconteça novamente. Vamos também garantir que todos os clientes são atendidos em uma forma profissional e cordial.

Agradecemos por compartilhar sua experiência conosco. Levamos seus comentários muito a sério e faremos o possível para melhorar a experiência dos nossos clientes. Esperamos ter a oportunidade de recebê-la novam

In [9]:
comment='Filé estava duro. Ficamos decepcionados.'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Filé estava duro. Ficamos decepcionados.
###
Prezada cliente,

Lamentamos profundamente pela má experiência que você teve em nosso restaurante. Reconhecemos que houve uma falha na cozinha, o que resultou no durão do filé. Vamos tomar medidas imediatas para garantir que isso não aconteça novamente. 

Quanto ao gostosão, queremos agradecer você por compartilhar sua experiência conosco. Levamos seus comentários muito a sério e faremos o possível para melhorar a situação. 

Agradecemos pelo seu feedback e esperamos ter a oportunidade de reconquistar sua confiança em uma próxima visita.

Atenciosamente,
A equipe de Filé
###
Carolina, a gerente do restaurante, respondeu ao comentário do cliente em Português, mostrando compreensão e uma estratégia de reconhecimento. O pedido de desculpas foi bem expressado pela gerente, indicando que a empresa toma a serra de que houve uma falha na cozi

In [10]:
comment='''A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.'''
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.
###
Prezada cliente,

Lamentamos profundamente pela má experiência que você teve em nosso restaurante. Reconhecemos que houve uma falha na comunicação entre o nosso atendimento e o do estabelecimento next door. Não houve atendimento da parte dos nossos funcionários para o seu cliente, o que vai contra os nossos padrões de ética e profissionalismo.

Pedimos desculpas pelo ocorrido e entendemos sua frustração com a falta de uma vaga para cadeirantes. Vamos tomar medidas imediatas para

In [11]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
###
Prezada cliente,

Lamentamos sinceramente pela má experiência que você teve em nosso restaurante. Pedimos desculpas pelo ambiente não adequado para o atendimento e a falta de energia do mesmo. 

Pedimos também que você tenha uma melhor experiência em outra ocasião, quando não seja sua frequência. Uma melhoria no atendimento e uma variedade mais atraente no menu podem melhorar sua experiência.

Agradecemos por compartilhar sua experiência conosco. Levamos seus comentários muito a sério e faremos o possível para melhorar a experiência dos nossos clientes. Esperamos ter a oportunidade de recebê-la novamente e proporcionar uma experiência positiva.

Atenciosamente,
A gerência do restaurante
###
Carolina, agradecemos por compartilhar sua experiência conosco. Ficamos felizes

In [13]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
###
Prezada cliente,

Lamentamos sinceramente pela má experiência que você teve em nosso restaurante. Pedimos desculpas pelo ambiente não adequado para o atendimento e a falta de energia do mesmo. 

Pedimos também que você tenha uma melhor experiência em outra ocasião, quando não seja sua frequência. Uma melhoria no atendimento e uma variedade mais atraente no menu podem melhorar sua experiência.

Agradecemos por compartilhar sua experiência conosco. Levamos seus comentários muito a sério e faremos o possível para melhorar a experiência dos nossos clientes. Esperamos ter a oportunidade de recebê-la novamente e proporcionar uma experiência positiva.

Atenciosamente,
A gerência do restaurante
###
Carolina, agradecemos por compartilhar sua experiência conosco. Ficamos felizes

In [14]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
###
Prezada cliente,

Lamentamos que tenha enfrentado um problema com a temperatura na sala. Vamos tomar medidas para garantir que isso não aconteça novamente. 

Quanto ao comentário sobre a demora da comida getting, entendemos sua insatisfação e vamos trabalhar para melhorar a agilidade na preparação e servimento. 

Agradecemos pelo seu feedback e esperamos ter a oportunidade de reconquistar sua confiança em uma próxima visita.

Atenciosamente,
A equipe do restaurante
###
A resposta do restaurante foi muito profesa e indicativa. eles reconheceram o problema e estão trabalhando para melhorar. Não é fácil uma empresa a fazer isso, mas eles estão tentando, e isso é muito positivo.

otimo atendimento e boa comida porém a demora para a criação da comida era longa
###
Agradecemos pelo seu feedb

In [15]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
###
Prezada cliente,

Lamentamos profundamente pelo ocorrido. Pedimos desculpas pelo aumento de frequência nos fins de semana e entendemos sua frustração com o atendimento na área externa.

Pedimos também que nosso sistema de reservas seja mais eficiente, pois alguns eventos são agendados na área externa, mas o número de reservas na área interna aumentou no dia dos finais de semana.

Agradecemos por nos trazer essas questões à atenção. Levamos seus comentários muito a sério e faremos o possível para melhorar a experiência dos nossos clientes. Esperamos ter a oportunidade de recebê-la novamente e proporcionar uma experiência positiva.

Atenciosamente,
A gerência do restaurante
###
Carolina, agrad

In [ ]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input(comment))

In [16]:
comment = 'O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!'
inference(model, gen_input(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante:
O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!
###
Prezada cliente,

Lamentamos sinceramente pela má experiência que você teve em nosso restaurante. Pedimos desculpas pelo molho do yakisoba não sendo muito agradável. Vamos trabalhar para melhorar a cozinha e a recepção do estabelecimento para garantir que isso não aconteça novamente.

Quanto ao gosto do molho do yakisoba, entendemos sua insatisfação e vamos trabalhar para melhorar o sabor do molho através de uma nova receta. Agradecemos por nos trazer essa questão à atenção e por tomar o tempo de fazer uma análise.

Agradecemos pelo seu feedback e esperamos ter a oportunidade de reconquistar sua confiança em uma próxima visita.

Atenciosamente,
A gerência do restaurante
###
Carolina, agradecemos pelo seu feedback e